In [1]:
'''
Date: 20230214
Aim: get the npz of two water figures
Author: Yike Xie
'''

'\nDate: 20230214\nAim: get the npz of two water figures\nAuthor: Yike Xie\n'

In [2]:
import numpy as np
import os
import cv2
import h5py

In [3]:
def read_image(fn):
    with h5py.File(fn, 'r') as f:
        print('Read metadata about excitation/emission lambdas')
        keys = ['excitationWavelength', 'emission']
        wls = {key: [] for key in keys}
        for key in keys:
            tmp = f['HAC_Image']['imageStruct']['protocol']['channel'][key]
            n_colors = tmp.size
            for i in range(n_colors):
                tmpi = f[f[tmp[i, 0]][0, 0]][:, 0]
                wl = int(tmpi.astype(dtype=np.uint8).tobytes().decode())
                wls[key].append(wl)
        wls['combo'] = []
        for i in range(n_colors):
            wls['combo'].append(
                (wls['excitationWavelength'][i], wls['emission'][i]),
                )

        print('Read image data')
        img = f['HAC_Image']['imageStruct']['data'][:, :, :]
    return {
        'data': img,
        'wavelengths': wls['combo'],
        'image': os.path.basename(fn).split('.')[0],
    }


def save_npz(img, wls, save_path, fn):
    img_data = {}

    img = img.astype(np.uint16)
    for i, wl in enumerate(wls):
        img_data['wls_{}_{}'.format(str(wl[0]), str(wl[1]))] = img[i].T

    np.savez_compressed(save_path + fn.split('.')[0] + '.npz', **img_data)

In [10]:
# transfer water .mat files to .npz files
fdn_mat = '/home/yike/phd/cancer_cells_img_seq/data/20220201_imaging/control/'
save_path = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'

fns = ['Water-1.mat', 'Water-2.mat', 'Cal-1.mat', 'Cal-2.mat', 'Cal-3.mat', 'Test-1.mat', 'Test-2.mat']

for fn in fns:
    print('get image date {}'.format(fn.split('.')[0]))
    data = read_image(os.path.join(fdn_mat, fn))
    img = data['data']
    wls = data['wavelengths']

    print('save figure data')
    save_npz(img, wls, save_path, fn)

get image date Water-1
Read metadata about excitation/emission lambdas
Read image data
save figure data
get image date Water-2
Read metadata about excitation/emission lambdas
Read image data
save figure data


In [4]:
# crop the background from water samples of each cell
def crop_npz(fdn_fig, fn_ctl, fns, fdn_save):
    print('load control npz...')
    ctl = np.load(os.path.join(fdn_fig, fn_ctl)) # load water npz
    
    ncells = len(fns)    
    for i, fn in enumerate(fns):
        cellname = fn.split('.')[0]
        print(f'{cellname}: {i+1}/{ncells}')

        print('load cell npz...')
        raw = np.load(fdn_fig + fn)
        seg = raw['segmentation']

        print('find ROI...')
        i0s = seg.any(axis=1).nonzero()[0]
        i1s = seg.any(axis=0).nonzero()[0]
        i00, i01 = i0s[0], i0s[-1] + 1
        i10, i11 = i1s[0], i1s[-1] + 1

        print('add margins...')
        if 'G7_S57' in fn:
            margin_px = 60
        else:
            margin_px = 30
        i00 = max(0, i00 - margin_px)
        i10 = max(0, i10 - margin_px)
        i01 = min(seg.shape[0], i01 + margin_px)
        i11 = min(seg.shape[1], i11 + margin_px)

        print('crop control...')
        img_dict = {}
        for key, val in ctl.items():
            if key == 'segmentation':
                continue
            img_dict[key] = val[i00: i01, i10: i11]

        print('save croped control to file...')
        np.savez_compressed(fdn_save + fn, **img_dict)

        print('done')

In [22]:
# crop background of water-1
fdn_fig = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fns = [i for i in os.listdir(fdn_fig) if 'Water' not in i]

fn_ctl = 'Water-1.npz'
fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_water_1/'

crop_npz(fdn_fig, fn_ctl, fns, fdn_save)

load control npz...
C6_S40: 1/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
I8_S72: 2/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J3_S8: 3/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H10_S97: 4/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J10_S99: 5/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
G11_S109: 6/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C7_S53: 7/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L3_S10: 8/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C5_S27: 9/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to 

save croped control to file...
done
G8_S70: 75/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N9_S90: 76/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H11_S110: 77/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L6_S49: 78/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N11_S116: 79/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H12_S123: 80/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C4_S14: 81/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
M9_S89: 82/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L7_S62: 83/121
load cell npz...
find ROI...
add margins...
crop contro

In [23]:
# crop background of water-2
fdn_fig = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fns = [i for i in os.listdir(fdn_fig) if 'Water' not in i]

fn_ctl = 'Water-2.npz'
fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_water_2/'

crop_npz(fdn_fig, fn_ctl, fns, fdn_save)

load control npz...
C6_S40: 1/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
I8_S72: 2/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J3_S8: 3/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H10_S97: 4/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J10_S99: 5/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
G11_S109: 6/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C7_S53: 7/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L3_S10: 8/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C5_S27: 9/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to 

save croped control to file...
done
G8_S70: 75/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N9_S90: 76/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H11_S110: 77/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L6_S49: 78/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N11_S116: 79/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H12_S123: 80/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C4_S14: 81/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
M9_S89: 82/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L7_S62: 83/121
load cell npz...
find ROI...
add margins...
crop contro

In [32]:
# check if size of backgrounds and cells are the same

fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_water_2/'

fdn_cell = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/cancer100cells/'
sig_fns = os.listdir(fdn_cell)
fdn_dis = os.path.join(fdn_cell, 'doublets/distant_cells')
dis_fns = os.listdir(fdn_dis)
fdn_db = os.path.join(fdn_cell, 'doublets/doublets')
db_fns = os.listdir(fdn_db)

mis_fns = []

for fn in os.listdir(fdn_save):
    shape_ctl = np.load(os.path.join(fdn_save, fn))['wls_325_414'].shape
    if fn in sig_fns:
        shape_cell = np.load(os.path.join(fdn_cell, fn))['wls_325_414'].shape
    elif fn in dis_fns:
        shape_cell = np.load(os.path.join(fdn_dis, fn))['wls_325_414'].shape
    elif fn in db_fns:
        shape_cell = np.load(os.path.join(fdn_db, fn))['wls_325_414'].shape
        
    if shape_ctl != shape_cell:
        print(fn)
        mis_fns.append(fn)

In [5]:
# crop background of test-1
fdn_fig = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fns = [i for i in os.listdir(fdn_fig) if '_' in i]

fn_ctl = 'Test-1.npz'
fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_test_1/'

crop_npz(fdn_fig, fn_ctl, fns, fdn_save)

load control npz...
C6_S40: 1/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
I8_S72: 2/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J3_S8: 3/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H10_S97: 4/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J10_S99: 5/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
G11_S109: 6/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C7_S53: 7/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L3_S10: 8/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C5_S27: 9/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to 

save croped control to file...
done
G8_S70: 75/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N9_S90: 76/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H11_S110: 77/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L6_S49: 78/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N11_S116: 79/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H12_S123: 80/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C4_S14: 81/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
M9_S89: 82/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L7_S62: 83/121
load cell npz...
find ROI...
add margins...
crop contro

In [6]:
# crop background of test-2
fdn_fig = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fns = [i for i in os.listdir(fdn_fig) if '_' in i]

fn_ctl = 'Test-2.npz'
fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_test_2/'

crop_npz(fdn_fig, fn_ctl, fns, fdn_save)

load control npz...
C6_S40: 1/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
I8_S72: 2/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J3_S8: 3/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H10_S97: 4/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J10_S99: 5/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
G11_S109: 6/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C7_S53: 7/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L3_S10: 8/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C5_S27: 9/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to 

save croped control to file...
done
G8_S70: 75/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N9_S90: 76/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H11_S110: 77/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L6_S49: 78/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N11_S116: 79/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H12_S123: 80/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C4_S14: 81/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
M9_S89: 82/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L7_S62: 83/121
load cell npz...
find ROI...
add margins...
crop contro

In [6]:
# crop background of cal-2
fdn_fig = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fns = [i for i in os.listdir(fdn_fig) if '_' in i]

fn_ctl = 'Cal-2.npz'
fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_cal_2/'

crop_npz(fdn_fig, fn_ctl, fns, fdn_save)

load control npz...
C6_S40: 1/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
I8_S72: 2/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J3_S8: 3/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H10_S97: 4/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J10_S99: 5/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
G11_S109: 6/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C7_S53: 7/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L3_S10: 8/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C5_S27: 9/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to 

save croped control to file...
done
G8_S70: 75/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N9_S90: 76/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H11_S110: 77/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L6_S49: 78/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N11_S116: 79/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H12_S123: 80/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C4_S14: 81/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
M9_S89: 82/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L7_S62: 83/121
load cell npz...
find ROI...
add margins...
crop contro

In [7]:
# crop background of cal-3
fdn_fig = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/figures/'
fns = [i for i in os.listdir(fdn_fig) if '_' in i]

fn_ctl = 'Cal-3.npz'
fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/segmentation/seg_cal_3/'

crop_npz(fdn_fig, fn_ctl, fns, fdn_save)

load control npz...
C6_S40: 1/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
I8_S72: 2/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J3_S8: 3/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H10_S97: 4/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
J10_S99: 5/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
G11_S109: 6/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C7_S53: 7/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L3_S10: 8/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C5_S27: 9/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to 

save croped control to file...
done
G8_S70: 75/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N9_S90: 76/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H11_S110: 77/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L6_S49: 78/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
N11_S116: 79/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
H12_S123: 80/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
C4_S14: 81/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
M9_S89: 82/121
load cell npz...
find ROI...
add margins...
crop control...
save croped control to file...
done
L7_S62: 83/121
load cell npz...
find ROI...
add margins...
crop contro